# pytorchvideo.data.Ucf101の使い方

pytorchvideo.dataを使ってUFC101を読み込む．

- https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#ucf101



# データセットの準備

UFC101はあらかじめダウンロードして展開済みであるとする．
- `/dataset/UCF101/video/`以下に，101クラスのサブディレクトリがある
- `/dataset/UCF101/ucfTrainTestlist/`以下に，UCF101のアノテーションファイルであるtrainlist0{1,2,3}.txtなどがある

## フォルダ構成

```bash
$ tree -I "*.avi" /dataset/UCF101
/dataset/UCF101
├── ucfTrainTestlist
│   ├── classInd.txt
│   ├── testlist01.txt
│   ├── testlist02.txt
│   ├── testlist03.txt
│   ├── trainlist01.txt
│   ├── trainlist02.txt
│   └── trainlist03.txt
└── video
    ├── ApplyEyeMakeup
    ├── ApplyLipstick
    ├── Archery
    ├── BabyCrawling
    ├── BalanceBeam
...
    ├── Typing
    ├── UnevenBars
    ├── VolleyballSpiking
    ├── WalkingWithDog
    ├── WallPushups
    ├── WritingOnBoard
    └── YoYo

$ head -5 /dataset/UCF101/ucfTrainTestlist/*
==> /dataset/UCF101/ucfTrainTestlist/classInd.txt <==
1 ApplyEyeMakeup
2 ApplyLipstick
3 Archery
4 BabyCrawling
5 BalanceBeam

==> /dataset/UCF101/ucfTrainTestlist/testlist01.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/testlist02.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/testlist03.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/trainlist01.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1

==> /dataset/UCF101/ucfTrainTestlist/trainlist02.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi 1

==> /dataset/UCF101/ucfTrainTestlist/trainlist03.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi 1
```


In [1]:
import torch

from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler

from pytorchvideo.data import Ucf101
from pytorchvideo.data import RandomClipSampler, make_clip_sampler

import pytorchvideo.transforms as videoTrans
import torchvision.transforms as visionTrans

import itertools
import os

argparseを真似たパラメータ設定．
こうしておくとすぐ本物のargpaseに移行できる．

In [2]:
class Args:
    def __init__(self):

        self.data_root = '/dataset/UCF101/'
        self.video_path_prefix = os.path.join(
            self.data_root, 'video')
        self.annotation_path = os.path.join(
            self.data_root, 'ucfTrainTestlist')

        self.clip_duration = 16 / 25  # 25FPSを想定して16枚分の時間（0.64秒）
        self.frames_per_clip = 16

        self.clips_per_video = 3  # only for ConstantClipsPerVideoSampler

        self.batch_size = 8
        self.num_workers = 1

args = Args()

# transform


バッチはdict形式なので，video, label, audioなどのそれぞれにtransformが設定できる
- ApplyTransformToKeyでkeyを指定して，video/label/audio用のtransformをそれぞれ設定


参考：https://pytorchvideo.org/docs/tutorial_classification



## video用のtransform

pytorchvideo.transformとtorchvision.transformが混在するので，
それぞれvideoTransとvisionTransという名前にしてimportしてある．
videoTransには動画用のtransformある．
画像にも動画にも使えるものはvisionTransを利用する．


- UniformTemporalSubsampleで固定枚数をサンプルする
    - datasetのclip_samplerには，秒単位でしか与えられないらしい．
    - 枚数指定ではfpsが異なる動画ではサンプルされる枚数も変わってしまう．そこで，ここで取得するフレーム数を揃える
    - UFC101のFPSは25が多いが30も存在する．
        - 25.00 fps: 10810  
        - 30 fps: 2510
    - 25fpsを仮定して16フレームを1 clipとすると，0.64秒．もし30fspなら約19枚分．したがって，
      - 25fpsの動画からは16フレームをサンプリングして，ここで16枚に間引く（つまり何もしない）
      - 30fpsの動画からは19フレームをサンプリングして，ここで16枚に間引く（19枚からどうやって16枚を抜き出すのかは，まだソースを見ていないので不明）
- 読み込んだフレーム型はfloat32だが値は0-255なので，255で割ってfloatにする．
- 読み込んだフレームのshapeは(C, T, H, W)．これがpytorchvideoでは一般的．
- Normalizeは画像フレーム単位で行うので，visionTransのものでも良さそうだが，shapeが問題．内部では以下のことをしている．
    - shapeを(C, T, H, W)から(T, C, H, W)に変更
    - vision.transform.Normalizeを適用
        - `Expected tensor to be a tensor image of size (..., C, H, W)`
          https://github.com/pytorch/vision/blob/183a722169421c83638e68ee2d8fc5bd3415c4b4/torchvision/transforms/functional.py#L320
    - shapeを(T, C, H, W)から(C, T, H, W)に戻す
    - https://github.com/facebookresearch/pytorchvideo/blob/e236b0bcaf81dcfb78b7a34bf234028eb3b85f21/pytorchvideo/transforms/transforms.py#L153
- 短い方を256画素程度に合わせてから，画像を224x224にリサイズする（action認識の一般的なやり方）
  - 学習時：RandomShortSideScaleなら厳密には256になっていないが（データ拡張のため），RandcomCropで224x224を切り出す
  - テスト時：ShortSideScaleなら256になる．これをCenterCropで224x224を切り出す



## label用のtransform

UCF101の学習用ラベルファイル（trainlist01.txtなど）には1から101までのラベルが付いているが，それがそのまま使われてしまう（なぜだ．．．）．
このままではindex範囲外というエラーが（不定期に）発生してしまうので，ここでラベルの値をtransformで0から100に修正する．

テスト用ファイルラベルファイル（testlist01.txtなど）にはラベルは含まれていないので，これをする必要はない．

## audio用のtransform

audioは使わないのでRemoveKeyで除去する．

UCF101には音声がない動画が約半数しかないため．
- 音声あり: 6837
- 音声なし: 6483




In [3]:
train_transform = visionTrans.Compose([
    videoTrans.ApplyTransformToKey(
        key="video",
        transform=visionTrans.Compose([
            videoTrans.UniformTemporalSubsample(
                args.frames_per_clip),
            visionTrans.Lambda(lambda x: x / 255.),
            videoTrans.Normalize((0.45, 0.45, 0.45),
                                 (0.225, 0.225, 0.225)),
            videoTrans.RandomShortSideScale(
                min_size=256, max_size=320,),
            visionTrans.RandomCrop(224),
            visionTrans.RandomHorizontalFlip(),
        ]),
    ),
    videoTrans.ApplyTransformToKey(
        key="label",
        # ラベルが1から101になっているので，1を引いて0から100にする
        transform=visionTrans.Lambda(lambda x: x - 1),
    ),
    videoTrans.RemoveKey("audio"),
])

val_transform = visionTrans.Compose([
    videoTrans.ApplyTransformToKey(
        key="video",
        transform=visionTrans.Compose([
            videoTrans.UniformTemporalSubsample(
                args.frames_per_clip),
            visionTrans.Lambda(lambda x: x / 255.),
            videoTrans.Normalize((0.45, 0.45, 0.45),
                                 (0.225, 0.225, 0.225)),
            videoTrans.ShortSideScale(256),
            visionTrans.CenterCrop(224),
        ]),
    ),

    videoTrans.RemoveKey("audio"),
])


# UCF101データセットオブジェクトの作成


- clip_sampler=RandomClipSampler(clip_duration=args.clip_duration)：あるビデオ中のクリップのサンプリングをランダムに行う．
    - 長さはclip_durationで指定．25fpsの動画から16フレームを抜き出すつもりなら，16/25=0.64にする（単位は秒）
- video_sampler=RandomSampler：どのビデオを使うのかはランダムに選択
- decode_audio=False：音声は使わない

## データの指定方法

- data_path：'ucfTrainTestlist/trainlist01.txt'などのアノテーションファイルを指定する．
- video_path_prefix：アノテーションファイルで指定されている動画ファイル名の前につけるprefix．

例：
`trainlist01.txt`で指定されているのは
```text
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
```
という形式で，実際にはファイルが
```
/dataset/UCF101/video/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
```
にあるなら，
`/dataset/UCF101/video/`が`video_path_prefix`になる．


In [4]:
# clip_sampler = RandomClipSampler(clip_duration=args.clip_duration)
clip_sampler = make_clip_sampler("random", args.clip_duration)
# clip_sampler = make_clip_sampler("uniform", args.clip_duration)
# clip_sampler = make_clip_sampler("constant_clips_per_video", args.clip_duration, args.clips_per_video)


train_set = Ucf101(
    data_path=os.path.join(args.annotation_path, 'trainlist01.txt'),
    video_path_prefix=args.video_path_prefix,
    clip_sampler=clip_sampler,
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=train_transform,
    )

val_set = Ucf101(
    data_path=os.path.join(args.annotation_path, 'testlist01.txt'),
    video_path_prefix=args.video_path_prefix,
    clip_sampler=clip_sampler,
    video_sampler=RandomSampler,
    decode_audio=False,
    transform=val_transform,
    )

num_classes = 101

動画ファイルが壊れていてclipがサンプルできない場合でも，1エポックで同じstepになることを保証するために，以下のラッパーを利用する．

In [5]:
# https://github.com/facebookresearch/pytorchvideo/blob/ef2d3a96bb939b12aa0f21fb467d2175b0f05e9f/tutorials/video_classification_example/train.py#L343

class LimitDataset(torch.utils.data.Dataset):
    """
    To ensure a constant number of samples are retrieved from the dataset we use this
    LimitDataset wrapper. This is necessary because several of the underlying videos
    may be corrupted while fetching or decoding, however, we always want the same
    number of steps per epoch.
    """

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        self.dataset_iter = itertools.chain.from_iterable(
            itertools.repeat(iter(dataset), 2)
        )

    def __getitem__(self, index):
        return next(self.dataset_iter)

    def __len__(self):
        return self.dataset.num_videos

このラッパーを使ってdataloaderオブジェクトを作成．

In [6]:
train_set = LimitDataset(train_set)
val_set = LimitDataset(val_set)

In [7]:
train_loader = DataLoader(train_set,
                          batch_size=args.batch_size,
                          drop_last=True,
                          num_workers=args.num_workers)
val_loader = DataLoader(val_set,
                        batch_size=args.batch_size,
                        drop_last=True,
                        num_workers=args.num_workers)


# data loaderの挙動

バッチを取り出して挙動を確認．
- バッチはdictでやってくるので，`batch['video']`と`batch['label']`で取り出す
- train_loaderでは，RandomClipSamplerならランダムなラベルが得られている．
- val_loaderでは，ラベル情報はないのでラベルは0ばかり

## clip samplerの影響

3種類あるclip samplerの比較．
- randomは各サンプルclipがランダム
- uniformはある1本の動画から同じ長さのclipをサンプルし続ける（1本あたりのclip数は未定）
- constant per clipは，ある1本の動画から，同数のclipをサンプルし続ける（1本あたりのclips数は固定，指定する）

batchのキーの解釈
- labelは各サンプルのラベル
- video_indexは動画ファイルのID
- clip_index
  - random: 未使用（すべて0）
  - uniform: 1エポック中で何番目のclipを表す番号？（何に使う？）
    - [リセットしてないので](https://github.com/facebookresearch/pytorchvideo/blob/5c34ca13956425c63533923eebaf7c3b57acc71c/pytorchvideo/data/clip_sampling.py#L160)，エポックが変わってもインクリメントし続けるかも
  - constant per clip: 各動画からサンプリングしたclipの番号（この場合は0, 1, 2）

random
```
batch 0
label      : [64 74 48 86  1 83 24 65]
video_index: [6166 7099 4700 8248  155 7959 2390 6268]
clip_index : [0 0 0 0 0 0 0 0]
batch 1
label      : [30 41 42 19 55 34 30 70]
video_index: [2911 3975 4081 1919 5278 3345 2970 6786]
clip_index : [0 0 0 0 0 0 0 0]
batch 2
label      : [74 70 42 58 51 32 24 60]
video_index: [7087 6719 4112 5576 4934 3092 2392 5774]
clip_index : [0 0 0 0 0 0 0 0]
batch 3
label      : [  5  53  37  43 100  27  95  23]
video_index: [ 552 5153 3580 4191 9489 2651 9069 2309]
clip_index : [0 0 0 0 0 0 0 0]
```

uniform
```
batch 0
label      : [ 9  9  9  9  9 31 31 31]
video_index: [ 950  950  950  950  950 2973 2973 2973]
clip_index : [0 1 2 3 4 5 6 7]
batch 1
label      : [31 31 31 31 31 98 98 98]
video_index: [2973 2973 2973 2973 2973 9332 9332 9332]
clip_index : [ 8  9 10 11 12 13 14 15]
batch 2
label      : [98 98 98 98 98 98 98 98]
video_index: [9332 9332 9332 9332 9332 9332 9332 9332]
clip_index : [16 17 18 19 20 21 22 23]
batch 3
label      : [44 44 44 44 44 44  2  2]
video_index: [4262 4262 4262 4262 4262 4262  241  241]
clip_index : [24 25 26 27 28 29 30 31]
```

constant per video
```
batch 0
label      : [49 49 49 71 71 71 24 24]
video_index: [4808 4808 4808 6875 6875 6875 2385 2385]
clip_index : [0 1 2 0 1 2 0 1]
batch 1
label      : [24 12 12 12 70 70 70 86]
video_index: [2385 1211 1211 1211 6735 6735 6735 8219]
clip_index : [2 0 1 2 0 1 2 0]
batch 2
label      : [86 86 76 76 76 72 72 72]
video_index: [8219 8219 7306 7306 7306 6909 6909 6909]
clip_index : [1 2 0 1 2 0 1 2]
batch 3
label      : [52 52 52 80 80 80 38 38]
video_index: [5027 5027 5027 7710 7710 7710 3662 3662]
clip_index : [0 1 2 0 1 2 0 1]
```



In [8]:
print('train')
for i, batch in enumerate(train_loader):
    if i == 0:
        print(batch.keys())
        print('batch shape', batch['video'].shape)
    print('batch', i)
    print('label      :', batch['label'].cpu().numpy())
    print('video_index:', batch['video_index'].cpu().numpy())
    print('clip_index :', batch['clip_index'].cpu().numpy())
    if i > 2:
        break

print('val')
for i, batch in enumerate(val_loader):
    if i == 0:
        print(batch.keys())
        print('batch shape', batch['video'].shape)
    print('batch', i)
    print('label      :', batch['label'].cpu().numpy())
    print('video_index:', batch['video_index'].cpu().numpy())
    print('clip_index :', batch['clip_index'].cpu().numpy())
    if i > 2:
        break


train
dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
batch shape torch.Size([8, 3, 16, 224, 224])
batch 0
label      : [59 16 41 83 96 19 66 56]
video_index: [5675 1608 3982 7993 9114 1872 6403 5395]
clip_index : [0 0 0 0 0 0 0 0]
batch 1
label      : [ 2 20 56 72 81 12 88 33]
video_index: [ 284 1969 5350 6904 7757 1241 8383 3245]
clip_index : [0 0 0 0 0 0 0 0]
batch 2
label      : [57 59 24 67  2 94 41 63]
video_index: [5437 5667 2392 6513  200 8937 3972 6085]
clip_index : [0 0 0 0 0 0 0 0]
batch 3
label      : [80 21 89 38 65 75 75 94]
video_index: [7689 2066 8518 3687 6274 7181 7248 8924]
clip_index : [0 0 0 0 0 0 0 0]
val
dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
batch shape torch.Size([8, 3, 16, 224, 224])
batch 0
label      : [-1 -1 -1 -1 -1 -1 -1 -1]
video_index: [3505 2081 1728 3105  781 1506 3428 1065]
clip_index : [0 0 0 0 0 0 0 0]
batch 1
label      : [-1 -1 -1 -1 -1 -1 -1 -1]
video_index: [ 2

データローダーのlenを確認する．

- 各ビデオから一つのクリップがサンプルされている
  - RandomClipSamplerならランダムにクリップをサンプリング

In [9]:
print('train set')
print('num batches', len(train_loader))
print('num samples', len(train_set))
print('num samples / batch_size = ', len(train_set) / args.batch_size)

train set
num batches 1192
num samples 9537
num samples / batch_size =  1192.125


In [10]:
print('val set')
print('num batches', len(val_loader))
print('num samples', len(val_set))
print('num samples / batch_size = ', len(val_set) / args.batch_size)

val set
num batches 472
num samples 3783
num samples / batch_size =  472.875


# 学習ループ

学習ループを想定して，dataloaderからバッチを取り出す

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
num_epochs = 2

for epoch in range(num_epochs):
    print("Epoch ", epoch)

    for batch_idx, batch in enumerate(train_loader):

        inputs = batch['video'].to(device)
        targets = batch['label'].to(device)

        print('batch ', batch_idx)
        print('current batch size', inputs.size(0))
        print('input shape', inputs.shape)
        print('target shape', targets.shape)

        if batch_idx > 2:
            break  # stop for demonstration

Epoch  0
batch  0
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  1
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  2
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  3
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
Epoch  1
batch  0
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  1
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  2
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
batch  3
current batch size 8
input shape torch.Size([8, 3, 16, 224, 224])
target shape torch.Size([8])
